In [1]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
import pyproj
# import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import h5py
# import pyhdf
# from pyhdf.SD import SD, SDC
# import re
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from PIL import Image
import scipy.stats as stats
import numpy.ma as ma

In [2]:
# Peru intact site coordinates (Hergoualch): -3.832483, -73.31418

In [3]:
# For intact site, using plot 1 section 1 in-situ water table data (units are cm)

In [4]:
# Note that dates will not match exactly due to ~monthly temporal resolution of data

In [5]:
qa_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_006063_20180725_20200831_02_T1_QA_PIXEL.TIF')



In [6]:
# Define red, nir, and swir bands
red_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_006063_20180725_20200831_02_T1_SR_B4.TIF')
nir_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_006063_20180725_20200831_02_T1_SR_B5.TIF')
swir_im = Image.open('/Users/nikaankoupaei/Downloads/LC08_L2SP_006063_20180725_20200831_02_T1_SR_B7.TIF')



In [7]:
# convert to pandas dataframe
red = pd.DataFrame(np.array(red_im))
nir = pd.DataFrame(np.array(nir_im))
swir = pd.DataFrame(np.array(swir_im))

In [8]:
# convert fill value to nans
red=red.replace(0,np.nan)
nir=nir.replace(0,np.nan)
swir=swir.replace(0,np.nan)

In [9]:
# filter through only valid values and replace non-valid values with nans 
mask_red = (red >= 7273) & (red <= 43636) 
mask_nir = (nir >= 7273) & (nir <= 43636)
mask_swir = (swir >= 7273) & (swir <= 43636)

red = red[mask_red]
nir = nir[mask_nir]
swir = swir[mask_swir]

In [10]:
# Multiply by scale factor and add offset
red = (0.0000275*red) - 0.2
nir = (0.0000275*nir) - 0.2
swir = (0.0000275*swir) - 0.2

In [11]:
# DELETE IF NEEDED

red = np.asarray(red)
nir = np.asarray(nir)
swir = np.asarray(swir)

In [12]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# # rows

# red_arr1 = np.delete(red,slice(5000,9999999),0)
# nir_arr1 = np.delete(nir,slice(5000,9999999),0)
# swir_arr1 = np.delete(swir,slice(5000,9999999),0)

# # # columns

# red_arr = np.delete(red_arr1,slice(5000,9999999),1)
# nir_arr = np.delete(nir_arr1,slice(5000,9999999),1)
# swir_arr = np.delete(swir_arr1,slice(5000,9999999),1)


In [13]:
red = red[1400:2400,3000:4000]
nir = nir[1400:2400,3000:4000]
swir = swir[1400:2400,3000:4000]

In [14]:
# check if shape is same for all
print(red.shape)
print(nir.shape)
print(swir.shape)

(1000, 1000)
(1000, 1000)
(1000, 1000)


In [15]:
# Calculate NDVI and assign to variable
ndvi=(nir-red)/(nir+red)

In [16]:
# Delete
ndvi_arr = np.asarray(ndvi) 


In [17]:
# filter ndvi values
ndvi_arr[ndvi_arr < 0] = np.nan
ndvi_arr[ndvi_arr > 1] = np.nan 

In [18]:
ndvi = pd.DataFrame(ndvi_arr)

In [19]:
# convert swir back to array
swir = np.asarray(swir)

In [20]:
# filter swir values
swir[swir < 0.01] = np.nan
swir[swir > 100] = np.nan

In [21]:
swir = pd.DataFrame(swir)

In [22]:
# Calculate STR and assign to variable
str0=((1-swir)**2)/(2*swir)

In [23]:
# THIS IS WHAT TO DO FOR FILTERING SWIR
SWIR=pd.DataFrame(np.asarray(swir).flatten())

In [24]:
SWIR=SWIR.rename(columns={0:'SWIR'})

In [25]:
# END WHAT TO DO FOR FILTERING FOR SWIR

In [26]:
# START WHAT TO DO FOR FILTERING FOR NDVI

In [27]:
NDVI=pd.DataFrame(np.asarray(ndvi).flatten())

In [28]:
NDVI=NDVI.rename(columns={0:'NDVI'})

In [29]:
# (DELETE IF NEEDED)
NDVI['STR'] = ((1-SWIR['SWIR'])**2)/(2*SWIR['SWIR']) 

In [30]:
# MAIN DATAFRAME (DELETE IF NEEDED)
DF = NDVI
DF

,NDVI,STR
0,0.259752,0.708565
1,0.257930,0.702933
2,0.242087,0.680121
3,0.293198,0.844582
4,0.404574,1.085019
...,...,...
999995,0.051224,0.457879
999996,0.070374,0.562249
999997,0.084797,0.633829
999998,0.070867,0.582133


In [31]:
x_wet=np.linspace(0,1,100)
x_dry=np.linspace(0,1,100)

In [32]:
# convert to array and flatten ndvi and str
ndvi_flat=np.asarray(ndvi).flatten()
str0_flat=np.asarray(str0).flatten()

In [33]:
# convert both to list
ndvi_list = ndvi_flat.tolist()
str0_list = str0_flat.tolist()

In [34]:
# calculate STR max list (change "before" equation below accordingly)
str_max_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(30.134*ndvi_list[i])+5.856
            str_max_list.append(before)
    else:
        str_max_list.append(np.nan) 

In [35]:
# calculate STR min list (change "before" equation below accordingly)
str_min_list=[]
for i in range(len(ndvi_list)):
    if ((30.134*ndvi_list[i])+5.856) > str0_list[i] and ((2.121*ndvi_list[i])-0.296) < str0_list[i]:
            before=(2.121*ndvi_list[i])-0.296
            str_min_list.append(before)
    else:
        str_min_list.append(np.nan)

In [36]:
str0_max_arr = np.asarray(str_max_list)
str0_min_arr = np.asarray(str_min_list)

In [37]:
str_max = str0_max_arr.reshape((red.shape[0], red.shape[1]))
str_min = str0_min_arr.reshape((red.shape[0], red.shape[1]))

In [38]:
qa_arr = np.asarray(qa_im)

In [39]:
qa_arr_sliced = qa_arr[1400:2400,3000:4000]

In [40]:
# DELETE IF NEEDED

# slice the dataframes to obtain only region of interest

# rows

# qa_arr111 = np.delete(qa_arr,slice(5000,9999999),0)

# # columns

# qa_arr_sliced = np.delete(qa_arr111,slice(5000,9999999),1)


In [41]:
qa_arr_sliced.shape

(1000, 1000)

In [42]:
qa = pd.DataFrame(qa_arr_sliced.flatten())

In [43]:
qa.shape

(1000000, 1)

In [44]:
df_qa=qa.rename(columns={0:'QA'})
df_qa

,QA
0,22280
1,22280
2,22280
3,22280
4,22280
...,...
999995,22280
999996,22280
999997,22280
999998,22280


In [45]:
OPTRAM = (str0 - str_min)/(str_max - str_min)

In [46]:
df_OPTRAM = pd.DataFrame(np.asarray(OPTRAM).flatten())

In [47]:
df_OPTRAM=df_OPTRAM.rename(columns={0:'OPTRAM'})
df_OPTRAM

,OPTRAM
0,0.033781
1,0.033778
2,0.035772
3,0.036108
4,0.029906
...,...
999995,0.085045
999996,0.087277
999997,0.087948
999998,0.089444


In [48]:
print(df_qa.shape)
print(df_OPTRAM.shape)

(1000000, 1)
(1000000, 1)


In [49]:
df_OPTRAM['QA'] = df_qa['QA']
df_OPTRAM

,OPTRAM,QA
0,0.033781,22280
1,0.033778,22280
2,0.035772,22280
3,0.036108,22280
4,0.029906,22280
...,...,...
999995,0.085045,22280
999996,0.087277,22280
999997,0.087948,22280
999998,0.089444,22280


In [50]:
# Setting all rows that are not clear (i.e. those that have cloudy pixels)

df_clear = df_OPTRAM.where(df_OPTRAM['QA'] == 21824)

In [51]:
df_clear

,OPTRAM,QA
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
999995,NaN,NaN
999996,NaN,NaN
999997,NaN,NaN
999998,NaN,NaN


In [52]:
OPTRAM_clear_array = np.asarray(df_clear['OPTRAM'])
OPTRAM_clear_array.shape

(1000000,)

In [53]:
OPTRAM_FINAL_CLOUD_MASKED = OPTRAM_clear_array.reshape((1000,1000))
OPTRAM_FINAL_CLOUD_MASKED.shape

(1000, 1000)

In [54]:
pd.DataFrame(OPTRAM_FINAL_CLOUD_MASKED).to_csv('/Users/nikaankoupaei/Downloads/ownCloud/20231222_OPTRAM_FILES_REALIZED_DIDNT_INLCUDE_VALID_RANGE_ENDS_FOR_LANDSAT/PERU_mDeg/NEW_FOR_20231222_cloud_masked_OPTRAM_20180725_hergoualch_mDEG_filtered.csv')


